In [12]:
from web3 import Web3
from ens import ENS
import pandas as pd


In [13]:

provider = "https://eth-mainnet.alchemyapi.io/v2/" + 'vDU8lgwuQUrLRkDZw0BumUfxW8Au27w_'
w3 = Web3(Web3.HTTPProvider(provider))
ns = ENS.fromWeb3(w3)
w3.eth.getTransactionCount('rohanio.eth')


/Users/rohan/opt/anaconda3/envs/diamond/lib/python3.9/site-packages/ens/main.py:285: FutureWarning: The function signature for resolver() will change in v6 to accept `name` as a the positional argument, over `normal_name`, and the method will instead normalize the name internally. You may migrate to using `name` by passing it in as a keyword, e.g. resolver(name="ensname.eth").
  warnings.warn(


44

In [ ]:

address = '0x6F351927b4A68D971C89E5D7C174438d52A34a60'
address = w3.toChecksumAddress(address)
name = ns.name(address)
print(name)
if address != ns.address(name):
  name = None




In [ ]:

address = '0x85ecca75f99aebc79e2a69540d95d9990c2aad2a'
address = address.lower()
x = w3.toChecksumAddress(address)

address = '0x1234...'
name = ns.reverse(address)
# Check to be sure the reverse record is correct.
if address != ns.address(name):
  name = None

In [ ]:

eoa_list = []
contract_list = []

# get address type (contract vs EOA)
result = w3.eth.get_code(x).hex() 
if result == '0x':
    eoa_list.append({'address': address})
else:
    contract_list.append({'address': address})

print(eoa_list, contract_list)

eoa_df = pd.DataFrame(eoa_list)
eoa_df.drop_duplicates(subset=['address'], inplace=True)

contract_df = pd.DataFrame(contract_list)
contract_df.drop_duplicates(subset=['address'], inplace=True)


def add_eoa_labels(url, conn):

    wallet_node_query = f"""
                            LOAD CSV WITH HEADERS FROM '{url}' AS wallets
                            MATCH (w:Wallet {{address: wallets.address}})
                            SET w:Eoa,
                                w.lastUpdateDt = datetime(apoc.date.toISO8601(apoc.date.currentTimestamp(), 'ms'))
                            return count(w)
                        """

    x = conn.query(wallet_node_query)
    print("Eoa Labels Addeds", x)

def add_contract_labels(url, conn):
    wallet_node_query = f"""
                        LOAD CSV WITH HEADERS FROM '{url}' AS wallets
                        MATCH (w:Wallet {{address: wallets.address}})
                        SET w:Contract,
                            w.lastUpdateDt = datetime(apoc.date.toISO8601(apoc.date.currentTimestamp(), 'ms'))
                        return count(w)
                    """

    x = conn.query(wallet_node_query)
    print("Contract Labels Added", x)

In [17]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

def query_gnosis_multisig(address):
    transport = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/gjeanmart/gnosis-safe-mainnet")
    client = Client(transport=transport, fetch_schema_from_transport=True)

    multisig_list = []
    query = gql(f""" {{
                        wallet(id: "{address}") {{
                            id
                            creator
                            network
                            stamp
                            hash
                            factory
                            mastercopy
                            owners
                            threshold
                            currentNonce
                            version
                        }}
                    }}""")

    result = client.execute(query)
    result = result['wallet']
    if result is not None:
        for owner in result['owners']:
            multisig_list.append({'multisig': address.lower(), 'address': owner.lower(), 'threshold': int(result['threshold'])})
    
    return multisig_list

multisig_df = pd.DataFrame(multisig_list)
multisig_df.drop_duplicates(subset=['address', 'multisig'], inplace=True)

def add_multisig_labels(url, conn):

    wallet_node_query = f"""
                            LOAD CSV WITH HEADERS FROM '{url}' AS wallets
                            MATCH (w:Wallet {{address: wallets.multisig}})
                            SET w:MultiSig,
                                w.lastUpdateDt = datetime(apoc.date.toISO8601(apoc.date.currentTimestamp(), 'ms'))
                            return count(w)
                        """

    x = conn.query(wallet_node_query)
    print("MultiSig Labels Addeds", x)

[{'multisig': '0x85ecca75f99aebc79e2a69540d95d9990c2aad2a', 'address': '0xc75446a6adaef73269dbdece73536977b2b639e0', 'threshold': 2}, {'multisig': '0x85ecca75f99aebc79e2a69540d95d9990c2aad2a', 'address': '0x97b9958facec9acb7adb2bb72a70172cb5a0ea7c', 'threshold': 2}]


In [4]:
import re

rex = re.compile("^[A-Za-z0-9_]{1,15}$")
bool(re.match(rex, 'optimiz3_academy'))

False

In [21]:
import requests

url = "https://eth-mainnet.g.alchemy.com/nft/v2/vDU8lgwuQUrLRkDZw0BumUfxW8Au27w_/getNFTs?owner=amphiboly.eth&contractAddresses[]=0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85&withMetadata=false"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"ownedNfts":[{"contract":{"address":"0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85"},"id":{"tokenId":"0x14c08e39a5e2b2e43828dbe48e4d7f676b976cfd6d65f284b1a7a74537fd7079"},"balance":"1"},{"contract":{"address":"0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85"},"id":{"tokenId":"0xac6bc6428530fe6f1cd4712fc5b1b27cc5f226bb1acea0c8756bd5e467d68d68"},"balance":"1"},{"contract":{"address":"0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85"},"id":{"tokenId":"0xaeeb6b2e33e85ce021e682d5d17d2dcb16cc7d0e5afb391628c0bccce8cdb5c8"},"balance":"1"},{"contract":{"address":"0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85"},"id":{"tokenId":"0xb56f21bd5ab9ad2d47ac1effcb49cfba33d3f9f224aab5818e4515a093913a40"},"balance":"1"}],"totalCount":4,"blockHash":"0x1e3bd4e35c7284d7bed024079ba0828d4ebafd5f17929160a285264ceac3b519"}


In [22]:
import json

x = json.loads(response.text)

x['ownedNfts'][0]

{'contract': {'address': '0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85'},
 'id': {'tokenId': '0x14c08e39a5e2b2e43828dbe48e4d7f676b976cfd6d65f284b1a7a74537fd7079'},
 'balance': '1'}

In [20]:
y = 9386473210270768446719447585399752119953056934940566112317246862703747494009
int(x['ownedNfts'][0]['id']['tokenId'], base=16) == y

True

In [8]:
url = "https://alchemy-sdk-core-example.com/vDU8lgwuQUrLRkDZw0BumUfxW8Au27w_/getNfts?owner=amphiboly.eth&contractAddresses[]=0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85&omitMetadata=false"

import requests
import json

headers = {"accept": "application/json"}

r = requests.get(url, headers=headers)
r.text


ConnectionError: HTTPSConnectionPool(host='alchemy-sdk-core-example.com', port=443): Max retries exceeded with url: /vDU8lgwuQUrLRkDZw0BumUfxW8Au27w_/getNfts?owner=amphiboly.eth&contractAddresses%5B%5D=0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85&omitMetadata=false (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9031dd8df0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [9]:
import requests

url = "https://alchemy-sdk-core-example.com/demo/getNfts?owner=rohanio.eth&contractAddresses[]=0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85&omitMetadata=false"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

ConnectionError: HTTPSConnectionPool(host='alchemy-sdk-core-example.com', port=443): Max retries exceeded with url: /demo/getNfts?owner=rohanio.eth&contractAddresses%5B%5D=0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85&omitMetadata=false (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9031db6c10>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [35]:
import rlp

print(hex(int(str(w3.toInt(w3.keccak(rlp.encode([w3.toChecksumAddress('0xA73dB69EA64315c44C970245C1f30A7bc765A7D0'), 557]))))[12:])))

0xc2e6701d7b8b836deeeb28e32fa2fb74bc77d0453d3de5323d694


In [9]:
from Crypto.Hash import keccak
from rlp.utils import decode_hex, encode_hex
import rlp
sha3_256 = lambda x: keccak.new(digest_bits=256, data=x).digest()

def to_string(value):
    if isinstance(value, bytes):
        return value
    if isinstance(value, str):
        return bytes(value, 'utf-8')
    if isinstance(value, int):
        return bytes(str(value), 'utf-8')

def sha3(seed):
    return sha3_256(to_string(seed))

def normalize_address(x, allow_blank=False):
    if allow_blank and x == '':
        return ''
    if len(x) in (42, 50) and x[:2] == '0x':
        x = x[2:]
    if len(x) in (40, 48):
        x = decode_hex(x)
    if len(x) == 24:
        assert len(x) == 24 and sha3(x[:20])[:4] == x[-4:]
        x = x[:20]
    if len(x) != 20:
        raise Exception("Invalid address format: %r" % x)
    return x

def mk_contract_address(sender, nonce):
    return sha3(rlp.encode([normalize_address(sender), nonce]))[12:]

In [14]:
x = '0x' + encode_hex(mk_contract_address('0xA73dB69EA64315c44C970245C1f30A7bc765A7D0', 557))

In [18]:
w3.eth.getCode(Web3.toChecksumAddress(x)).hex()

'...'

In [19]:
w3.eth.getCode(Web3.toChecksumAddress('0x4BB9E7D221fC35234a08cCdcb38D47D53Fb1973e')).hex()

'0x'

In [113]:
import pandas as pd
import numpy as np

lol = [{'school_id': 1, 'state_code': '!@sc_1', 'subjects': "english maths chemistry"},
{'school_id': 2, 'state_code': '))sc_2', 'subjects': "english physics chemistry"},
{'school_id': 3, 'state_code': '!@sc_2', 'subjects': "maths biology"},
{'school_id': 4, 'state_code': 'sc_2)_', 'subjects': "french"},
{'school_id': 5, 'state_code': 'sc_1#@', 'subjects': "social_studies maths literature"},
{'school_id': 6, 'state_code': 'sc_10#@', 'subjects': "english  maths history"}]

df = pd.DataFrame(lol)



In [114]:

sub_count = lambda x: len(x.split())
df = df[df['subjects'].apply(sub_count) >= 3]

del_chars = ''.join(c for c in map(chr, range(1114111)) if not c.isalnum())
del_map = str.maketrans('', '', del_chars)
alpha = lambda x: x.translate(del_map)
df.loc[:, 'state_code'] = df['state_code'].apply(alpha)

subjects = lambda x: x.split()
df.loc[:, 'subjects'] = df['subjects'].apply(subjects)

df = df.explode('subjects')
df = df[df['subjects'].isin(['english', 'maths', 'physics', 'chemistry'])]
df.drop(columns=['school_id'], inplace=True)

df = df.groupby(['state_code', 'subjects'])['subjects'].count().reset_index(name="count")
df = df.pivot(index='state_code', columns='subjects').reset_index()
df.columns = df.columns.droplevel(0)
df.rename(columns={'':'state_code'}, inplace=True)
df = df.rename_axis(None, axis=1)
df = df.fillna(0.0)
m = df.select_dtypes(np.number)
df[m.columns]= m.round().astype('Int64')

df.columns = ['state_code', 'english', 'maths', 'physics', 'chemistry']
df = df.sort_values(by=['state_code'], ascending=True)

In [115]:
df

,state_code,english,maths,physics,chemistry
0,sc1,1,1,2,0
1,sc10,0,1,1,0
2,sc2,1,1,0,1
